# Interdisciplinary Team Strategy

**Mental Model:** Different domain experts collaborate on complex problems

**Date:** [Fill in]  
**Author:** Leif Haven Martinson  

## Concept

Complex problems often require expertise from multiple disciplines:
- Engineer + Designer + Business person
- Doctor + Ethicist + Policy expert
- Economist + Sociologist + Data scientist

This notebook simulates an interdisciplinary team where each agent has a specific domain expertise and contributes their perspective.

## How It Works

1. **Problem Statement** - Present a complex problem
2. **Expert Analysis** - Each expert analyzes from their domain
3. **Integration** - Synthesize insights into unified solution
4. **Refinement** (optional) - Experts critique the synthesis and refine

## Use Cases

- Strategic planning
- Product development
- Policy decisions
- Research design
- Complex problem-solving

In [ ]:
# Setup
import sys
sys.path.append('../code')

from harness import (
    llm_call,
    llm_call_stream,
    ExperimentConfig,
    ExperimentResult,
    get_tracker
)
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

print("✅ Setup complete")

## Configuration

**CUSTOMIZE HERE:** Define your interdisciplinary team

In [ ]:
# ========================================
# TEAM CONFIGURATION - EDIT THESE
# ========================================

# Model configuration
PROVIDER = DEFAULT_PROVIDER
MODEL = DEFAULT_MODEL
TEMPERATURE = 0.7

# Number of refinement rounds after initial synthesis
REFINEMENT_ROUNDS = 1  # 🔧 0 = no refinement, 1+ = iterative improvement

# ========================================
# DEFINE YOUR EXPERT TEAM
# ========================================
# Each expert has:
# - name: Short identifier
# - role: Their domain/discipline
# - perspective: What they focus on
# - system_prompt: Their detailed expertise

EXPERT_TEAM = [
    {
        "name": "Technical Lead",
        "role": "Software Engineer",
        "perspective": "Technical feasibility, architecture, scalability",
        "system_prompt": """You are a senior software engineer with 15 years of experience.
Focus on:
- Technical feasibility and implementation complexity
- System architecture and design patterns
- Scalability, performance, and maintainability
- Potential technical risks and mitigation strategies

Analyze problems through a technical lens, considering what's actually buildable."""
    },
    {
        "name": "UX Designer",
        "role": "User Experience Designer",
        "perspective": "User needs, usability, accessibility",
        "system_prompt": """You are a UX designer focused on human-centered design.
Focus on:
- User needs, pain points, and goals
- Usability and user experience
- Accessibility and inclusive design
- User research and testing approaches

Analyze problems from the user's perspective, ensuring solutions actually serve people."""
    },
    {
        "name": "Business Strategist",
        "role": "Business Strategy",
        "perspective": "Market fit, ROI, business model",
        "system_prompt": """You are a business strategist with MBA and startup experience.
Focus on:
- Market opportunity and competitive landscape
- Business model and revenue potential
- ROI and resource allocation
- Go-to-market strategy

Analyze problems through a business lens, ensuring solutions create value."""
    },
    {
        "name": "Data Scientist",
        "role": "Data Science & Analytics",
        "perspective": "Data-driven insights, metrics, experimentation",
        "system_prompt": """You are a data scientist specializing in analytics and ML.
Focus on:
- What data is needed and available
- Key metrics to track success
- Experimental design and A/B testing
- Statistical validity and data quality

Analyze problems data-first, ensuring decisions are measurable and evidence-based."""
    },
    
    # Add more experts as needed:
    # {
    #     "name": "Security Expert",
    #     "role": "Cybersecurity",
    #     "perspective": "Security, privacy, compliance",
    #     "system_prompt": "You are a cybersecurity expert..."
    # },
]

# ========================================
# INTEGRATION PROMPT
# ========================================
# How the integrator synthesizes expert insights

INTEGRATOR_PROMPT = """You are a project lead synthesizing insights from an interdisciplinary team.

Your job:
1. Review each expert's analysis
2. Identify key insights and potential conflicts
3. Synthesize into a coherent, actionable solution
4. Balance competing priorities (technical, UX, business, data)
5. Propose concrete next steps

Provide a clear, integrated solution that respects all perspectives."""

print(f"✅ Team configured:")
print(f"   - {len(EXPERT_TEAM)} experts")
print(f"   - Refinement rounds: {REFINEMENT_ROUNDS}")
print(f"\n👥 Your expert team:")
for expert in EXPERT_TEAM:
    print(f"   - {expert['name']} ({expert['role']}): {expert['perspective']}")

## Define Problem

**CUSTOMIZE HERE:** Set the problem your team will tackle

In [ ]:
# ========================================
# PROBLEM STATEMENT - EDIT THIS
# ========================================

PROBLEM = """Our company wants to build a personalized AI tutor for high school students.

Context:
- Target: US high school students (grades 9-12)
- Subjects: Math, Science, English, History
- Platform: Web and mobile app
- Timeline: Launch MVP in 6 months
- Resources: 5-person team, $500K budget

Question:
What should our MVP include, and what's the best approach to build and launch it?

Consider:
- Technical approach (which AI models, architecture)
- User experience (what features are essential)
- Business model (pricing, customer acquisition)
- Success metrics (how do we measure if it works)
"""

# Alternative problems to try:
# 
# "We need to reduce our cloud infrastructure costs by 40% without impacting user experience."
# 
# "Should we open-source our core ML model or keep it proprietary?"
# 
# "Design a data collection strategy for improving our recommendation system."

print("📋 Problem defined:")
print(PROBLEM[:200] + "...")

## Run Expert Analysis

Each expert analyzes the problem from their domain

In [ ]:
import time

# Track timing and tokens
start_time = time.time()
total_tokens_in = 0
total_tokens_out = 0

# Collect expert analyses
expert_analyses = []

print(f"\n{'='*80}")
print(f"🎯 PROBLEM: {PROBLEM[:100]}...")
print(f"{'='*80}\n")

for expert in EXPERT_TEAM:
    print(f"\n{'─'*80}")
    print(f"👤 {expert['name']} ({expert['role']})")
    print(f"{'─'*80}\n")
    
    # Create expert-specific prompt
    expert_prompt = f"{expert['system_prompt']}\n\n"
    expert_prompt += f"Problem:\n{PROBLEM}\n\n"
    expert_prompt += f"Provide your analysis from the {expert['role']} perspective. "
    expert_prompt += f"Focus on: {expert['perspective']}\n\n"
    expert_prompt += "Your analysis:"
    
    # Get expert's analysis (streaming for visibility)
    full_text = ""
    response = None
    
    for chunk in llm_call_stream(expert_prompt, provider=PROVIDER, model=MODEL, temperature=TEMPERATURE):
        if isinstance(chunk, str):
            print(chunk, end="", flush=True)
            full_text += chunk
        else:
            response = chunk
    
    print("\n")
    
    # Store analysis
    expert_analyses.append({
        "expert": expert['name'],
        "role": expert['role'],
        "analysis": response.text
    })
    
    total_tokens_in += response.tokens_in or 0
    total_tokens_out += response.tokens_out or 0

print(f"\n✅ Expert analysis complete!")
print(f"   Experts consulted: {len(expert_analyses)}")

## Synthesize Integrated Solution

Integrate all expert perspectives into a unified solution

In [ ]:
print(f"\n{'='*80}")
print(f"🔄 SYNTHESIS: Integrating Expert Insights")
print(f"{'='*80}\n")

# Build synthesis prompt
synthesis_prompt = f"{INTEGRATOR_PROMPT}\n\n"
synthesis_prompt += f"Original Problem:\n{PROBLEM}\n\n"
synthesis_prompt += "Expert Analyses:\n\n"

for analysis in expert_analyses:
    synthesis_prompt += f"{analysis['expert']} ({analysis['role']}):\n"
    synthesis_prompt += f"{analysis['analysis']}\n\n"
    synthesis_prompt += "-" * 60 + "\n\n"

synthesis_prompt += """Based on all expert analyses:

1. Synthesize key insights
2. Identify any conflicts or tradeoffs
3. Propose an integrated solution
4. Provide concrete next steps

Integrated solution:"""

# Get synthesis (streaming)
full_text = ""
response = None

for chunk in llm_call_stream(synthesis_prompt, provider=PROVIDER, model=MODEL, temperature=0.3):
    if isinstance(chunk, str):
        print(chunk, end="", flush=True)
        full_text += chunk
    else:
        response = chunk

print("\n")

integrated_solution = response.text
total_tokens_in += response.tokens_in or 0
total_tokens_out += response.tokens_out or 0

print(f"✅ Synthesis complete!")

## Refinement Rounds (Optional)

Experts review the synthesis and suggest refinements

In [ ]:
if REFINEMENT_ROUNDS > 0:
    current_solution = integrated_solution
    
    for round_num in range(REFINEMENT_ROUNDS):
        print(f"\n{'='*80}")
        print(f"🔄 REFINEMENT ROUND {round_num + 1}")
        print(f"{'='*80}\n")
        
        refinements = []
        
        # Each expert reviews and suggests improvements
        for expert in EXPERT_TEAM:
            print(f"\n{'─'*80}")
            print(f"👤 {expert['name']} - Refinement")
            print(f"{'─'*80}\n")
            
            refinement_prompt = f"{expert['system_prompt']}\n\n"
            refinement_prompt += f"Original Problem:\n{PROBLEM}\n\n"
            refinement_prompt += f"Proposed Solution:\n{current_solution}\n\n"
            refinement_prompt += f"Review this solution from your {expert['role']} perspective.\n"
            refinement_prompt += "Provide specific suggestions for improvement or concerns.\n\n"
            refinement_prompt += "Your feedback:"
            
            # Get refinement
            full_text = ""
            response = None
            
            for chunk in llm_call_stream(refinement_prompt, provider=PROVIDER, model=MODEL, temperature=TEMPERATURE):
                if isinstance(chunk, str):
                    print(chunk, end="", flush=True)
                    full_text += chunk
                else:
                    response = chunk
            
            print("\n")
            
            refinements.append({
                "expert": expert['name'],
                "feedback": response.text
            })
            
            total_tokens_in += response.tokens_in or 0
            total_tokens_out += response.tokens_out or 0
        
        # Integrate refinements
        print(f"\n{'='*80}")
        print(f"🔄 Integrating Refinements")
        print(f"{'='*80}\n")
        
        integration_prompt = f"{INTEGRATOR_PROMPT}\n\n"
        integration_prompt += f"Current Solution:\n{current_solution}\n\n"
        integration_prompt += "Expert Refinements:\n\n"
        
        for refinement in refinements:
            integration_prompt += f"{refinement['expert']}:\n{refinement['feedback']}\n\n"
        
        integration_prompt += """Incorporate the expert feedback to improve the solution.
        
Refined solution:"""
        
        # Get refined solution
        full_text = ""
        response = None
        
        for chunk in llm_call_stream(integration_prompt, provider=PROVIDER, model=MODEL, temperature=0.3):
            if isinstance(chunk, str):
                print(chunk, end="", flush=True)
                full_text += chunk
            else:
                response = chunk
        
        print("\n")
        
        current_solution = response.text
        total_tokens_in += response.tokens_in or 0
        total_tokens_out += response.tokens_out or 0
    
    final_solution = current_solution
else:
    final_solution = integrated_solution
    print("\n⏭️  Skipping refinement (REFINEMENT_ROUNDS = 0)")

# Calculate total time
total_time = time.time() - start_time

print(f"\n{'='*80}")
print(f"✅ COMPLETE")
print(f"{'='*80}")
print(f"\n📊 Stats:")
print(f"   Experts: {len(EXPERT_TEAM)}")
print(f"   Refinement rounds: {REFINEMENT_ROUNDS}")
print(f"   Total time: {total_time:.1f}s")
print(f"   Total tokens: {total_tokens_in + total_tokens_out:,}")
print(f"   (In: {total_tokens_in:,}, Out: {total_tokens_out:,})")

## Summary View

Review the complete process

In [ ]:
print("\n" + "="*80)
print("📋 FINAL SOLUTION")
print("="*80 + "\n")
print(final_solution)
print("\n" + "="*80)

## Save Results

In [ ]:
# Save to file
import json
from datetime import datetime
from pathlib import Path

# Create results directory
results_dir = Path("../experiments/interdisciplinary_team")
results_dir.mkdir(parents=True, exist_ok=True)

# Create result object
result = {
    "problem": PROBLEM,
    "experts": [{"name": e["name"], "role": e["role"]} for e in EXPERT_TEAM],
    "expert_analyses": expert_analyses,
    "final_solution": final_solution,
    "refinement_rounds": REFINEMENT_ROUNDS,
    "stats": {
        "total_time_s": total_time,
        "total_tokens": total_tokens_in + total_tokens_out,
        "model": MODEL,
        "provider": PROVIDER
    },
    "timestamp": datetime.now().isoformat()
}

# Save JSON
filename = results_dir / f"result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w') as f:
    json.dump(result, f, indent=2)

print(f"\n💾 Saved to: {filename}")

## Next Steps

### Try Different Configurations

1. **Change the problem** - Test different complexity levels
2. **Add/remove experts** - Try 2, 5, or 7 experts
3. **Adjust expert roles** - Domain experts for your field
4. **Increase refinement rounds** - See if iteration helps
5. **Compare to single expert** - Does interdisciplinary help?

### Experiment Ideas

- **Product team**: Engineer + Designer + PM + Marketer
- **Medical team**: Doctor + Nurse + Researcher + Ethicist  
- **Research team**: Theorist + Experimentalist + Statistician + Domain expert
- **Policy team**: Economist + Sociologist + Legal expert + Implementer